## Abstract

This tutorial is about community detection. For better understanding and practice, firstly it introduces a very helpful and effective library `igraph` and how to use it. Then, it talks about the community and detection algorithms.

## Library
### Installation
> _ “igraph is a collection of network analysis tools with the emphasis on efficiency, portability and ease of use. igraph is open source and free. igraph can be programmed in R, Python and C/C++.”_

` $ pip install python-igraph`



In [328]:
from igraph import *

For making sure that you have successfully installed the igraph library, let's create your first igraph graph.
### Create a Graph
To create a graph, it should be a igraph object named `Graph`. Using `Graph()` to generate a graph manually.  

In [329]:
g = Graph()
print(g)
# summary(g)

IGRAPH U--- 0 0 --


In [330]:
g_full = Graph.Full(3)
print(g_full)

IGRAPH U--- 3 3 --
+ edges:
0--1 0--2 1--2


`igraph` use integer to donates the vertex ID and edges are specidied by pairs of integers. You could use edge list to create a graph.


In [331]:
g1 = Graph([(0,1), (0,2), (2,3), (3,4), (4,2), (2,5), (5,0), (6,3), (5,6)])
print(g1)

IGRAPH U--- 7 9 --
+ edges:
0 -- 1 2 5     2 -- 0 3 4 5   4 -- 2 3       6 -- 3 5
1 -- 0         3 -- 2 4 6     5 -- 0 2 6


**Note**:In general, you should use **`summary()`** instead of **`print()`** when working interactively with large graphs because printing the edge list of a graph with millions of vertices and edges could take quite a lot of time.

In daily use, we often process and analyse the outer dataset so that we need to load graph from files. The simplest way is `load()` which is a function tries to infer the appropriate format from the file extension. You will need to specify the mode of the graph(`directed=False`) if you want to load an undirected graph.

In [332]:
g_facebook = load("facebook.txt",directed=False)
summary(g_facebook)

IGRAPH U--- 4039 88234 -- 


### Vertices and Edges 
You can use **`Graph.add_vertices()`** (i.e., the **`add_vertices()`** method of the Graph class)  to add the given number of vertices to the graph. 

_What should be noticed is that igraph uses integer vertex IDs starting from zero and the vertices ID in igraph must be continuously. _

In [333]:
g.add_vertices(3)
print(g)

IGRAPH U--- 3 0 --


Now our graph has three vertices but no edges, so let’s add some edges as well! 

You can add edges by passing a list of edge to ** `Graph.add_edges()` **(i.e., the **`add_edges()`)**. 

You have to refer to the existing vertice and make sure the integers in the edge list exist in the current graph. Otherwise, there would be an error.

In [334]:
g.add_edges([(0,1), (1,2)])
print(g)
summary(g)

IGRAPH U--- 3 2 --
+ edges:
0--1 1--2
IGRAPH U--- 3 2 -- 


In [335]:
# when you add the edge list contains nonexist node(invalid vertex id), you program would throw an error. 
g.add_edges([(0,1), (0,2), (2,3), (3,4), (4,2), (2,5), (5,0), (6,3), (5,6)])

InternalError: Error at type_indexededgelist.c:272: cannot add edges, Invalid vertex id

Cool! Let's make this graph bigger. We can use **`Graph.get_edgelist()`** to get the edges. This function return a list a edges so that it could be index using slice in python.

In [336]:
g.add_vertices(4)
g.add_edges([(0,1), (0,2), (2,3), (3,4), (4,2), (2,5), (5,0), (6,3), (5,6)])
g.get_edgelist()[0:5]

[(0, 1), (1, 2), (0, 1), (0, 2), (2, 3)]

In [337]:
print(g)

IGRAPH U--- 7 11 --
+ edges:
0 -- 1 1 2 5     2 -- 0 1 3 4 5   4 -- 2 3         6 -- 3 5
1 -- 0 0 2       3 -- 2 4 6       5 -- 0 2 6


### Attributes

Every igraph, vertex and edge can behave as dictionaries, which means that you can add key-value pairs to them. In this case the key represents the name of the attribute and the value represent the value of the corresponding attribuates.

The number of the value of each attribute do not need to be equal to the number of vertice or edges. When the value more then the vertices or edges, the surplus will be cut. While, when it is insufficient，the value would be assigned by circle.

Then how can we get the attributes and how to assign values? 

Every Graph object contains two special members called **vs** and **es**, which are classes representing a sequence of vertices and edges in the graph.

For vertex instance, it has index and attributes(); for edge instance, it has source(the souce vertex of that edge), target(the target vertex of that edge) and attributes() also.


In [338]:
g["name"] = 'students for 15688'
print(g)

IGRAPH U--- 7 11 -- students for 15688
+ attr: name (g)
+ edges:
0 -- 1 1 2 5     2 -- 0 1 3 4 5   4 -- 2 3         6 -- 3 5
1 -- 0 0 2       3 -- 2 4 6       5 -- 0 2 6


In [339]:
g.vs

In [340]:
g.es

In [341]:
g.vs["name"] = ["Alice", "Bob", "Claire", "Dennis", "Esther", "Frank", "George"]
g.vs["score"] = [99, 100, 70, 60, 89, 73, 95]
g.vs["gender"] = ["f", "m", "f", "m", "f", "m", "m"]
print(g)

IGRAPH UN-- 7 11 -- students for 15688
+ attr: name (g), gender (v), name (v), score (v)
+ edges (vertex names):
 Alice -- Bob, Bob, Claire, Frank             Esther -- Claire, Dennis
   Bob -- Alice, Alice, Claire                 Frank -- Alice, Claire, George
Claire -- Alice, Bob, Dennis, Esther, Frank   George -- Dennis, Frank
Dennis -- Claire, Esther, George


**vs** and **es** are both iterable, so we can use loop to assign values or modify values.

In [342]:
for v in g.vs:
    attribute = v.attributes()
    if attribute["score"] > 90:
        v["letter"] = 'A' 
    elif attribute["score"] > 80:
        v["letter"] = 'B' 
    else:
        v["letter"] = 'C' 
    print(v.attributes())

{'name': 'Alice', 'score': 99, 'gender': 'f', 'letter': 'A'}
{'name': 'Bob', 'score': 100, 'gender': 'm', 'letter': 'A'}
{'name': 'Claire', 'score': 70, 'gender': 'f', 'letter': 'C'}
{'name': 'Dennis', 'score': 60, 'gender': 'm', 'letter': 'C'}
{'name': 'Esther', 'score': 89, 'gender': 'f', 'letter': 'B'}
{'name': 'Frank', 'score': 73, 'gender': 'm', 'letter': 'C'}
{'name': 'George', 'score': 95, 'gender': 'm', 'letter': 'A'}


In [343]:
g.es["master"] = [False, False, True, True, True, False, True, False, False]
g.es["weight"] = [1, 2, 3, 2, 1, 6, 4, 5, 2]
print(g)

IGRAPH UNW- 7 11 -- students for 15688
+ attr: name (g), gender (v), letter (v), name (v), score (v), master (e),
  weight (e)
+ edges (vertex names):
 Alice -- Bob, Bob, Claire, Frank             Esther -- Claire, Dennis
   Bob -- Alice, Alice, Claire                 Frank -- Alice, Claire, George
Claire -- Alice, Bob, Dennis, Esther, Frank   George -- Dennis, Frank
Dennis -- Claire, Esther, George


In [344]:
for e in g.es:
    print("Edge tuple", e.tuple)
    attribute = e.attributes()
    if attribute["weight"] > 4:
        e["relation"] = 'close' 
    elif attribute["weight"] > 2:
        e["relation"] = 'genral' 
    else:
        e["relation"] = 'apart' 
    print("Edge attributes", e.attributes())

Edge tuple (0, 1)
Edge attributes {'master': False, 'weight': 1, 'relation': 'apart'}
Edge tuple (1, 2)
Edge attributes {'master': False, 'weight': 2, 'relation': 'apart'}
Edge tuple (0, 1)
Edge attributes {'master': True, 'weight': 3, 'relation': 'genral'}
Edge tuple (0, 2)
Edge attributes {'master': True, 'weight': 2, 'relation': 'apart'}
Edge tuple (2, 3)
Edge attributes {'master': True, 'weight': 1, 'relation': 'apart'}
Edge tuple (3, 4)
Edge attributes {'master': False, 'weight': 6, 'relation': 'close'}
Edge tuple (2, 4)
Edge attributes {'master': True, 'weight': 4, 'relation': 'genral'}
Edge tuple (2, 5)
Edge attributes {'master': False, 'weight': 5, 'relation': 'close'}
Edge tuple (0, 5)
Edge attributes {'master': False, 'weight': 2, 'relation': 'apart'}
Edge tuple (3, 6)
Edge attributes {'master': False, 'weight': 1, 'relation': 'apart'}
Edge tuple (5, 6)
Edge attributes {'master': False, 'weight': 2, 'relation': 'apart'}


You can simply delete an attribute using `del`

In [345]:
del g.vs["letter"]
for v in g.vs:
    print(v.attributes())

{'name': 'Alice', 'score': 99, 'gender': 'f'}
{'name': 'Bob', 'score': 100, 'gender': 'm'}
{'name': 'Claire', 'score': 70, 'gender': 'f'}
{'name': 'Dennis', 'score': 60, 'gender': 'm'}
{'name': 'Esther', 'score': 89, 'gender': 'f'}
{'name': 'Frank', 'score': 73, 'gender': 'm'}
{'name': 'George', 'score': 95, 'gender': 'm'}


### Visualization

You need to install `cairo` to do visualization. For instruction, see 
[https://www.cairographics.org/download/](https://www.cairographics.org/download/).


When you want to visualize the graph, you could just use `plot()`. This function has many parameters that allow you to get pretty picture of your graph. See more on http://igraph.org/r/doc/plot.common.html.

**Note**:_If you use python3(eg. python3.6), you need to edit the file which is in ~/anaconda3/lib/python3.6/site-packages/igraph/drawing/__init__.py like the following so that you can plot the graph in jupyter notebook._
<img src="edit.jpg">


Now you can see what you create above.

In [346]:
plot(g,bbox=(200,200))

The plot has many parameters,see for [http://igraph.org/r/doc/plot.common.html](http://igraph.org/r/doc/plot.common.html)
For example, you would like to show the label of each vertex and make the width of the edge different according the weight attribute of edges. Besides, the plot figure is better to be (200,200).  

In [347]:
g.vs["label"] = g.vs["name"]
plot(g,edge_width = g.es['weight'],bbox=(200,200),vertex_color="yellow")

For convience, we would like to use the following visualization style to visualize the graph uniformly.

In [348]:
color = ClusterColoringPalette(100)
visual_style = {}
visual_style["vertex_size"] = 10
visual_style["margin"] = 20
visual_style["bbox"] = (300, 300)

### Get Information
Using the following funciton can get some usful informaiton about the graph.

In [349]:
# find the neighbors
for v in g.vs:
    print(v.neighbors())

[igraph.Vertex(<igraph.Graph object at 0x1117489a8>, 1, {'name': 'Bob', 'score': 100, 'gender': 'm', 'label': 'Bob'}), igraph.Vertex(<igraph.Graph object at 0x1117489a8>, 1, {'name': 'Bob', 'score': 100, 'gender': 'm', 'label': 'Bob'}), igraph.Vertex(<igraph.Graph object at 0x1117489a8>, 2, {'name': 'Claire', 'score': 70, 'gender': 'f', 'label': 'Claire'}), igraph.Vertex(<igraph.Graph object at 0x1117489a8>, 5, {'name': 'Frank', 'score': 73, 'gender': 'm', 'label': 'Frank'})]
[igraph.Vertex(<igraph.Graph object at 0x1117489a8>, 0, {'name': 'Alice', 'score': 99, 'gender': 'f', 'label': 'Alice'}), igraph.Vertex(<igraph.Graph object at 0x1117489a8>, 0, {'name': 'Alice', 'score': 99, 'gender': 'f', 'label': 'Alice'}), igraph.Vertex(<igraph.Graph object at 0x1117489a8>, 2, {'name': 'Claire', 'score': 70, 'gender': 'f', 'label': 'Claire'})]
[igraph.Vertex(<igraph.Graph object at 0x1117489a8>, 0, {'name': 'Alice', 'score': 99, 'gender': 'f', 'label': 'Alice'}), igraph.Vertex(<igraph.Graph obj

In [350]:
# find a vertex 
g.vs.find('name'== 'Alice')

igraph.Vertex(<igraph.Graph object at 0x1117489a8>, 0, {'name': 'Alice', 'score': 99, 'gender': 'f', 'label': 'Alice'})

In [351]:
# get the degree of vertices
g.vs.degree()

[4, 3, 5, 3, 2, 3, 2]

In [352]:
# compute the shortest path
g.shortest_paths()

[[0, 1, 1, 2, 2, 1, 2],
 [1, 0, 1, 2, 2, 2, 3],
 [1, 1, 0, 1, 1, 1, 2],
 [2, 2, 1, 0, 1, 2, 1],
 [2, 2, 1, 1, 0, 2, 2],
 [1, 2, 1, 2, 2, 0, 1],
 [2, 3, 2, 1, 2, 1, 0]]

## What is "Comminity"?

A precise definition of what a "__comminity__" really is does not exist yet. One of most widely accepted and used definitions is that given by Newman and Gievan(2004):

> _"A community is a subgraph containing nodes which are more densely linked to each other than to the rest of the graph or equivalently, a graph has a comminity structure if the number of links into any subgraph is higher than the number of links between those subgraphs."_


## Comminity Detection Algorithms

### Dataset
The dataset we used in the following is from facebook named [ego-Facebook](https://snap.stanford.edu/data/index.html) which is downloaded from [SNAP(Stanford Network Analysis Project)](https://snap.stanford.edu/index.html).This dataset consists of the friends list collected by facebook. The completed dataset is an undirected graph and has 4030 nodes and 88,234 edges. For ease of presentation, we use a subgraph of it which contains 100 nodes and all the edegs among this 100 nodes.

In [353]:
g_large = Graph.Read_Edgelist('facebook.txt',directed=False)
g = g_large.subgraph(range(400,500))
plot(g, **visual_style)

### Edge Betweenness

You may hear about vertex betweenness. The Girvan–Newman algorithm extends this definition to the case of edges, defining the "edge betweenness" of an edge as the number of shortest paths between pairs of nodes that run along it. So the edge betweenness mesures the edge centrality.

Edge betweenness detection is a method proposed by Girvan and Newman, it's main idea is by progressively remove some edges that has high edge betweenness. Because that the edge connected two communities are likely to have high edge betweenness compared with other edges.

Because this algorithm needs to compute edges's betweenness for every iteration, it is slow. Now let's see the clustering result for our dataset.

In [130]:
cl = g.community_edge_betweenness()
cls = cl.as_clustering().membership
g.vs['cls'] = cls
visual_style["vertex_color"] = [color.get(cls) for cls in g.vs["cls"]]
plot(g, **visual_style)

### Fast Greedy

Fast greedy algorithm is a hierarchical approach. Different from the edge between algoriithm, it is bottom-up instead top-down. At the beginning, every vertex is a community, and communities are merged in greedy manner. For each merge, the algorithm tried to maximize the current modularity.

This method is fast compared with others while it's drawback is that communities below a given size threshold will always be merged with neighboring communities. So that this method is always used to be the first approximation commnunities. 

In [131]:
cl = g1.community_fastgreedy()
cls = cl.as_clustering().membership
g.vs['cls'] = cls
visual_style["vertex_color"] = [color.get(cls) for cls in g.vs["cls"]]
plot(g, **visual_style)

### Walktrap

Form the name you may guess this method is based on random walks. Bingo! 

The walktrap algorithm's principle is that when you randomly walk on the graph, you are more likely walk within one community because that there are few edges the leads you out. It is like fast greedy algorithm and it merges communities during the random walk. It is a bottom-up method and will generate a dendrogram which leaves are vertices. Therefore, you can use a threshold to select where to cut the dendrogram using the modularity score.

According to the original publication, this method is a little bit slower than the fast greedy algorithm but it is more accurate.

In [132]:
cl = g.community_walktrap()
cls = cl.as_clustering().membership
g.vs['cls'] = cls
visual_style["vertex_color"] = [color.get(cls) for cls in g.vs["cls"]]
plot(g, **visual_style)

### Spinglass

Spinglass is a method based on the [**Potts model**](https://en.wikipedia.org/wiki/Potts_model) which is from statistical pyhsis. The gernal idea is that initially we set c spin states, which at end would be the comminities, and each vertex can be in ont of these states. The edges represent which pairs of vertices would prefer to stay in the same spin stata and which prefer to have different spin states. 

This method is not fast and the result is not deterministic. Also it should be noticed is that the method could not be applied to unconnected graph. Like the following example shows.

In [133]:
cl = g.community_spinglass()
cls = cl.as_clustering().membership
g.vs['cls'] = cls
visual_style["vertex_color"] = [color.get(cls) for cls in g.vs["cls"]]
plot(g, **visual_style)

InternalError: Error at clustertool.cpp:286: Cannot work with unconnected graph, Invalid value

In [316]:
g_connect = g
delete_vertices = []
for v in g_connect.vs:
    if v.degree() < 2:
        delete_vertices.append(v)
g_connect.delete_vertices(delete_vertices)
     
cl = g_connect.community_spinglass()
cls = cl.membership
g_connect.vs['cls'] = cls
visual_style["vertex_color"] = [color.get(cls) for cls in g_connect.vs["cls"]]
plot(g_connect, **visual_style)

### Leading Eigenvector

Leading eigenvector is a top-down hierarchical approach. It is to find densely connected subgraphs in a graph by calculating the leading non-negative eigenvector of the modularity matrix of the graph.This method splits the graph into two parts in each step which is determined by the leading eigenvector of the community matrix. In this way, the splited two parts yield a significant increase in the modularity.

Due to the eigenvector calculations involved, it might not work on degenerate graphs where the ARPACK eigenvector solver is unstable. On non-degenerate graphs, it is likely to yield a higher modularity score than the fast greedy method, although it is a bit slower.

In [323]:
cl = g.community_leading_eigenvector()
cls = cl.membership
g.vs['cls'] = cls
visual_style["vertex_color"] = [color.get(cls) for cls in g.vs["cls"]]

plot(g, **visual_style)

### Label Propagation

Label Propagation is relatively simple method. At the beginning, each node has one of the k labels(k is the initial condiguration which is decided randomly). Then, the method runs an iteration to reassign labels to nodes to make each node takes the most frequent label of its neighbors in a synchronous manner. The iteration stops when all labels of nodes are the most frequent labels in there neighborhood.

This method itself is very fast, while because the initialization is random, the result could be different every time. Therefore, to build a consensus labeling, one may need to run this method a large number of times.

In [327]:
cl = g.community_label_propagation()
cls = cl.membership
g.vs['cls'] = cls
visual_style["vertex_color"] = [color.get(cls) for cls in g.vs["cls"]]
plot(g, **visual_style)

## Analysis

From the detection showing above, we can see that in social network like facebook, there indeed exists "Six Degrees of Separation" so most of the people are in a big community and inside this big community there are many small communities seperated by the connections between people. We are easily to see the relationship and potential similarity through the visualization of the community detection. 

## Refernce
1. Community: https://en.wikipedia.org/wiki/Community_structure
2. igraph: http://igraph.org
3. cairo: https://cairographics.org
4. Girvan Newman Algorithm: https://en.wikipedia.org/wiki/Girvan%E2%80%93Newman_algorithm
5. Walktrap Algorithm: Pascal Pons, Matthieu Latapy: [Computing communities in large networks using random walks](http://arxiv.org/abs/physics/0512106)
6. Spinglass Algorithm: [J. Reichardt and S. Bornholdt: Statistical Mechanics of Community Detection, Phys. Rev. E, 74, 016110 (2006)](http://arxiv.org/abs/cond-mat/0603718)
7. Leading Eigenvector Algorithm: [MEJ Newman: Finding community structure using the eigenvectors of matrices, arXiv:physics/0605087](https://arxiv.org/pdf/physics/0605087.pdf)
8. Label Propagation Algorithm: https://en.wikipedia.org/wiki/Label_Propagation_Algorithm